In [ ]:
%load_ext autoreload
%autoreload 2

import sys

In [ ]:
import os
import array

import pandas
import numpy

import plotly.express
import librosa
from matplotlib import pyplot as plt

from software.dataset.hussain2021 import load_data, load_meta
from firmware.process import GravitySplitter

In [ ]:

#dataset_path = './data/hx5kkkbr3j-1'
combined_path = './data2/hussain2021_accelerometer_brush_manual.parquet'
samplerate = 50
lower = 2.0
upper = 5.0

In [ ]:
acc_re = pandas.read_parquet(combined_path)
data_columns = ['acc_x', 'acc_y', 'acc_z', 'elapsed']
index_columns = ['filename', 'time']
meta_columns = list(set(acc_re.columns) - set(data_columns) - set(index_columns))
acc_re

In [ ]:
filename_meta =  acc_re.reset_index().groupby('filename').first()[meta_columns]
filename_meta.head(3)

In [ ]:
acc_re.reset_index().filename.nunique()

In [ ]:
def make_spectrogram(df, sr=50, hop_length=16, columns=None):
    if columns is None:
        columns = ['acc_x', 'acc_y', 'acc_z']
    df = df.sort_values('elapsed')
    df = df.dropna(subset=columns)
    
    n_fft = 4*hop_length

    # Make combined XYZ spectrogram, concat on frequency axis
    ss = []
    for c in columns:
        v = df[c].values
        S = librosa.stft(v, n_fft=n_fft, hop_length=hop_length)
        S = numpy.abs(S)
        S = librosa.amplitude_to_db(S)
        #S = S[1:,:] # drop DC
        ss.append(S)

    o = numpy.concatenate(ss)
    return o

mm = acc_re
specs = mm.groupby('filename').apply(make_spectrogram, include_groups=False).to_frame().rename(columns={0:'spectrogram'})
specs;

In [ ]:

for filename, spec in specs.iterrows():
    #print(filename)
    fig, ax = plt.subplots(1, figsize=(20, 5))
    S = spec.spectrogram
    N = numpy.quantile(S, q=0.10, axis=1, keepdims=True)
    #S = S - N
    #print(S.shape, N.shape)
    librosa.display.specshow(ax=ax, data=S)


In [ ]:
def to_spectrum(S):
    s = numpy.mean(S, axis=1)
    o = pandas.Series(s, index=numpy.arange(len(s)))
    return o

def compute_spectrums(df, sr, metric='motion', axes=['x', 'y', 'z']):
    dfs = []
    
    for axis in axes:
        c = f'{metric}_{axis}'

        freqs = librosa.fft_frequencies(n_fft=4*16, sr=50)
    
        specs = df.groupby('filename').apply(make_spectrogram, include_groups=False, columns=[c]).to_frame().rename(columns={0:'spectrogram'})
        ss = specs.spectrogram.apply(to_spectrum)
        ss.columns = freqs

        # Normalize
        N = numpy.mean(ss.values, axis=1, keepdims=1)
        ss = pandas.DataFrame(ss-N, columns=ss.columns, index=ss.index)
        
        sf = pandas.melt(ss.reset_index(), id_vars='filename', var_name='frequency')
        sf['axis'] = axis
        
        dfs.append(sf)

    out = pandas.concat(dfs)
    return out

spectrums = compute_spectrums(mm, sr=samplerate, metric='acc')
#spectrums

fig = plotly.express.line(spectrums,
    facet_row='axis',
    color='filename',
    x='frequency',
    y='value',
    height=600,
    width=1000,
    title='Toothbrushing spectrum - Hussain 2021 dataset',
    labels={
        "frequency": "Frequency (Hz)",
        "value": "Energy (dB)",
     },
)

fig.add_vrect(x0=lower, x1=upper, line_width=0, fillcolor="green", opacity=0.2, layer='below')
fig.update_layout(showlegend=False)